In [ ]:
# https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/

In [13]:

%pip install pandas
%pip install numpy
%pip install matplotlib
%pip install gensim
%pip install spacy
%pip install nltk
%pip install re
%pip install logging
%pip install warnings


ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


     ---------------------------------------- 0.0/96.0 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/96.0 kB ? eta -:--:--
     ---------------------------- --------- 71.7/96.0 kB 653.6 kB/s eta 0:00:01
     -------------------------------------- 96.0/96.0 kB 779.9 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [24 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 14, in <module>
        File "C:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\setuptools\__init__.py", line 7, in <module>
          import _distutils_hack.override  # noqa: F401
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\_distutils_hack\override.py", line 1, in <module>
          __import__('_distutils_hack').do_override()
        File "C:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\_distutils_hack\__init__.py", line 70, in do_override
          ensure_local_distutils()
        File "C:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\_distutils_hack\__init__.py", lin

In [1]:
import sys
# !{sys.executable} -m spacy download en
import re, numpy as np, pandas as pd
from pprint import pprint

# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

# NLTK Stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])

%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

# Load SpaCy English model
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Function to lemmatize the text using SpaCy
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for text in texts:
        doc = nlp(" ".join(text))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#Chargement des données
data = pd.read_csv("../../Donnees/analyse_texte/csv/cancer_sport_analyse.csv")
data2 = pd.read_csv("../../Donnees/analyse_texte/csv/cancer_fasting_analyse.csv")
data3 = pd.read_csv("../../Donnees/analyse_texte/csv/cancer_cannabis.csv")
tweets_df = pd.concat([data,data2,data3])

tweets_df.shape

(52916, 17)

In [3]:
tweets_df.head()

,Fichier,ID,Nb retweet,Nb like,Nb réponses,Nb citations,Hashtags,Texte,Mots,Phrases,Tags POS,Sentiment,Polarité,Subjectivité,Phrases nominales,Texte corrigé,Texte tokénizé
0,721916579533950976.json,721916579533950976,0,0,0,0.0,"['#CANCER', '#SPORT', '#PARENTING', '#SCHOOL',...",Is artificial turf giving athletes #cancer ? -...,"['Is', 'artificial', 'turf', 'giving', 'athlet...","[Sentence(""Is artificial turf giving athletes ...","[('Is', 'VBZ'), ('artificial', 'JJ'), ('turf',...","Sentiment(polarity=-0.6, subjectivity=1.0)",-0.6,1.0000,"['artificial turf', 'athletes # cancer', 'spor...",Is artificial turf giving athletes #cancer ? -...,"['Is', 'artificial', 'turf', 'giving', 'athlet..."
1,238304774825603072.json,238304774825603072,0,1,0,0.0,"['#CANCER', '#SPORT']",Frankel's victory at York made me feel '20 yea...,"['Frankel', ""'s"", 'victory', 'at', 'York', 'ma...","[Sentence(""Frankel's victory at York made me f...","[('Frankel', 'NNP'), (""'s"", 'POS'), ('victory'...","Sentiment(polarity=0.5, subjectivity=0.5)",0.5,0.5000,"['frankel', ""'s victory"", 'york', ""'20 years"",...",Fraenkel's victory at Work made me feel '20 ye...,"['Frankel', ""'s"", 'victory', 'at', 'York', 'ma..."
2,1007390700726575104.json,1007390700726575104,0,4,1,0.0,"['#SPORT', '#CANCER']",READ THIS! What an absolutely brilliant piece ...,"['READ', 'THIS', 'What', 'an', 'absolutely', '...","[Sentence(""READ THIS!""), Sentence(""What an abs...","[('READ', 'VB'), ('THIS', 'NN'), ('What', 'WP'...","Sentiment(polarity=0.5, subjectivity=0.5375)",0.5,0.5375,"['read this', 'brilliant piece', '# sport trib...",READ THIS! That an absolutely brilliant piece ...,"['READ', 'THIS', '!', 'What', 'an', 'absolutel..."
3,668965073889697795.json,668965073889697795,0,0,0,0.0,"['#CANCER', '#STROKE', '#SPORT']","MJA InSight News: #Cancer survivors, exercise ...","['MJA', 'InSight', 'News', 'Cancer', 'survivor...","[Sentence(""MJA InSight News: #Cancer survivors...","[('MJA', 'NNP'), ('InSight', 'NNP'), ('News', ...","Sentiment(polarity=0.0, subjectivity=0.0)",0.0,0.0000,"['mja insight', '# cancer survivors', 'atopic ...","MJA night News: #Cancer survivor, exercise bar...","['MJA', 'InSight', 'News', ':', '#', 'Cancer',..."
4,1098650711049555968.json,1098650711049555968,2,1,0,0.0,"['#HEALTH', '#CANCER', '#SPORT']",When #health #cancer &amp; #sport are inextric...,"['When', 'health', 'cancer', 'amp', 'sport', '...","[Sentence(""When #health #cancer &amp; #sport a...","[('When', 'WRB'), ('health', 'NN'), ('cancer',...","Sentiment(polarity=0.6000000000000001, subject...",0.6,0.9000,"['# health # cancer', '# sport', 'amazing', 'd...",When #health #cancer &amp; #sport are inextric...,"['When', '#', 'health', '#', 'cancer', '&', 'a..."


In [4]:
import re

#Suppression des liens commençant par http et https
def remove_url(text):
    if isinstance(text, str):
        return re.sub(r'https?:\S*', '', text)
    else:
        return text
tweets_df['Texte corrigé']=tweets_df['Texte corrigé'].apply(remove_url)
def remove_url1(text):
    if isinstance(text, str):
        return re.sub(r'http?:\S*', '', text)
    else:
        return text
tweets_df['Texte corrigé']=tweets_df['Texte corrigé'].apply(remove_url1)

In [5]:
def sent_to_words(sentences):
    for sent in sentences:
        if isinstance(sent, str):
            sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
            sent = re.sub('\s+', ' ', sent)  # remove newline chars
            sent = re.sub("\'", "", sent)  # remove single quotes
            sent = gensim.utils.simple_preprocess(str(sent), deacc=True)
            yield(sent)
        else:
            yield []

# Convert to list
data = tweets_df['Texte corrigé'].values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1])

<>:4: SyntaxWarning: invalid escape sequence '\S'
<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\S'
<>:5: SyntaxWarning: invalid escape sequence '\s'
C:\Users\User\AppData\Local\Temp\ipykernel_23280\166417147.py:4: SyntaxWarning: invalid escape sequence '\S'
  sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
C:\Users\User\AppData\Local\Temp\ipykernel_23280\166417147.py:5: SyntaxWarning: invalid escape sequence '\s'
  sent = re.sub('\s+', ' ', sent)  # remove newline chars


[['is', 'artificial', 'turf', 'giving', 'athletes', 'cancer', 'port', 'printing', 'school', 'turf', 'grass', 'lawn']]


In [6]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# !python3 -m spacy download en  # run in terminal once
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]
    return texts_out

data_ready = process_words(data_words)  # processed Text Data!

In [15]:
# Create Dictionary
id2word = corpora.Dictionary(data_ready)

# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data_ready]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=15,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=100)

pprint(lda_model.print_topics())


[(0,
  '0.106*"grow" + 0.105*"thing" + 0.077*"free" + 0.059*"start" + 0.058*"legal" '
  '+ 0.056*"long" + 0.053*"illegal" + 0.043*"trial" + 0.040*"happy" + '
  '0.029*"first"'),
 (1,
  '0.000*"hang" + 0.000*"association" + 0.000*"ebb" + 0.000*"ply_watch" + '
  '0.000*"force" + 0.000*"fool" + 0.000*"thcnation" + 0.000*"lead" + '
  '0.000*"let_parma" + 0.000*"drops_daily"'),
 (2,
  '0.252*"cancer" + 0.140*"cure" + 0.109*"medical" + 0.053*"health" + '
  '0.049*"sure" + 0.032*"medicine" + 0.024*"tumour" + 0.022*"treat" + '
  '0.017*"watch" + 0.017*"benefit"'),
 (3,
  '0.154*"cancer" + 0.142*"study" + 0.077*"treatment" + 0.063*"thc" + '
  '0.048*"tumor" + 0.048*"people" + 0.035*"brain" + 0.032*"reduce" + '
  '0.029*"mmj" + 0.028*"show"'),
 (4,
  '0.078*"cancer" + 0.058*"great" + 0.052*"drug" + 0.049*"plant" + '
  '0.035*"time" + 0.033*"die" + 0.032*"extract" + 0.030*"check" + '
  '0.029*"chemotherapy" + 0.028*"food"'),
 (5,
  '0.000*"hang" + 0.000*"association" + 0.000*"ebb" + 0.000*"ply_wa

In [16]:
# Compute coherence score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_ready, dictionary=id2word)
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

# Print topics
pprint(lda_model.print_topics())

Coherence Score:  0.44431329662619207
[(0,
  '0.106*"grow" + 0.105*"thing" + 0.077*"free" + 0.059*"start" + 0.058*"legal" '
  '+ 0.056*"long" + 0.053*"illegal" + 0.043*"trial" + 0.040*"happy" + '
  '0.029*"first"'),
 (1,
  '0.000*"hang" + 0.000*"association" + 0.000*"ebb" + 0.000*"ply_watch" + '
  '0.000*"force" + 0.000*"fool" + 0.000*"thcnation" + 0.000*"lead" + '
  '0.000*"let_parma" + 0.000*"drops_daily"'),
 (2,
  '0.252*"cancer" + 0.140*"cure" + 0.109*"medical" + 0.053*"health" + '
  '0.049*"sure" + 0.032*"medicine" + 0.024*"tumour" + 0.022*"treat" + '
  '0.017*"watch" + 0.017*"benefit"'),
 (3,
  '0.154*"cancer" + 0.142*"study" + 0.077*"treatment" + 0.063*"thc" + '
  '0.048*"tumor" + 0.048*"people" + 0.035*"brain" + 0.032*"reduce" + '
  '0.029*"mmj" + 0.028*"show"'),
 (4,
  '0.078*"cancer" + 0.058*"great" + 0.052*"drug" + 0.049*"plant" + '
  '0.035*"time" + 0.033*"die" + 0.032*"extract" + 0.030*"check" + '
  '0.029*"chemotherapy" + 0.028*"food"'),
 (5,
  '0.000*"hang" + 0.000*"asso

In [17]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = pd.concat([sent_topics_df, pd.DataFrame([int(topic_num), round(prop_topic,4), topic_keywords]).T], ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    contents = contents.reset_index(drop=True)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_ready)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,11,0.6889,"help, cancer, day, new, cannabisoil, stop, fight, amp, rush, breakthetaboo","[artificial, turf, give, athlete, cancer, port, print, school, turf, lawn]"
1,1,12,0.8667,"cannabis, cancer, cod, oil, despoil, pain, cbd, weed, marijuana, epilepsy","[fraenkel, victory, work, feel, year, well, trainer, air, cecil, battle, cancer, port]"
2,2,4,0.7398,"cancer, great, drug, plant, time, die, extract, check, chemotherapy, food","[read, absolutely, brilliant, piece, sport, tribe, survive, cancer, suffer, rude, honor, privile..."
3,3,3,0.8133,"cancer, study, treatment, thc, tumor, people, brain, reduce, mmj, show","[night, news, cancer, survivor, exercise, barrier, topic, dermatitis, bilingualism, stroke, spor..."
4,4,2,0.6587,"cancer, cure, medical, health, sure, medicine, tumour, treat, watch, benefit","[health, cancer, sport, inextricably, link, amazing, youngster, still, stay, play, soccer, diagn..."
5,5,11,0.5653,"help, cancer, day, new, cannabisoil, stop, fight, amp, rush, breakthetaboo","[help, honor, step, cancer, journey, give, hope, lady, cancer, event, dubaiculture]"
6,6,10,0.6889,"cancer, cannabinoid, cell, kill, child, research, growth, pot, woman, inhibit","[charity_ruin, cancer, research, baffle]"
7,7,12,0.8833,"cannabis, cancer, cod, oil, despoil, pain, cbd, weed, marijuana, epilepsy","[oil, list, usa_drug_list, ff, cancer, cod, pain, sport]"
8,8,10,0.5739,"cancer, cannabinoid, cell, kill, child, research, growth, pot, woman, inhibit","[cheshirehour, look, charity, work, cancer, child, sport, contact]"
9,9,11,0.7667,"help, cancer, day, new, cannabisoil, stop, fight, amp, rush, breakthetaboo","[seat, solution, bad, condition, train, give, sport, charity, cancer, fundraising]"


In [18]:
# Display setting to show more characters in column
pd.options.display.max_colwidth = 100

sent_topics_sorteddf_mallet = pd.DataFrame()
sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet,
                                             grp.sort_values(['Perc_Contribution'], ascending=False).head(1)],
                                            axis=0)

# Reset Index
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Representative Text"]

# Show
sent_topics_sorteddf_mallet.head(10)

,Topic_Num,Topic_Perc_Contrib,Keywords,Representative Text
0,0,0.7333,"grow, thing, free, start, legal, long, illegal, trial, happy, first","[destroy, imagine, doctor, fired_hospital, close_pension, systems_brake, child, wait, long, inhe..."
1,1,0.0667,"hang, association, ebb, ply_watch, force, fool, thcnation, lead, let_parma, drops_daily","[speech, inspire, generation, athlete, sport, inspiration]"
2,2,0.8833,"cancer, cure, medical, health, sure, medicine, tumour, treat, watch, benefit","[post, thought, cancer, bongfresh, bone, doctor, health, fitness, mmm, meditate, medical, fact, ..."
3,3,0.8963,"cancer, study, treatment, thc, tumor, people, brain, reduce, mmj, show","[study, show, hour, period, fast, hour, increase, longevity, decrease, incidence, disease, inclu..."
4,4,0.8445,"cancer, great, drug, plant, time, die, extract, check, chemotherapy, food","[check, distance, time, time, sport, ultrarunne, cancer, chemotherapy]"
5,8,0.8444,"cancer, patient, mme, find, life, journey, legalize, meet, science, become","[cancer, patient, credit, save, life, local, cancer]"
6,10,0.8833,"cancer, cannabinoid, cell, kill, child, research, growth, pot, woman, inhibit","[cannabinoid, kill, cancer, cell, cell_proptosis, corner, pot]"
7,11,0.8444,"help, cancer, day, new, cannabisoil, stop, fight, amp, rush, breakthetaboo","[give, fight, rare, form, cancer, fight, cancer]"
8,12,0.9067,"cannabis, cancer, cod, oil, despoil, pain, cbd, weed, marijuana, epilepsy","[buy_nonthc_pure, cbd, oil, states_store, cod, cancer, cbdlife, cbdheal, cbdlife]"


In [19]:
# Sentence Coloring of N Sentences
def topics_per_document(model, corpus, start=0, end=1):
    corpus_sel = corpus[start:end]
    dominant_topics = []
    topic_percentages = []
    for i, corp in enumerate(corpus_sel):
        topic_percs, wordid_topics, wordid_phivalues = model[corp]
        dominant_topic = sorted(topic_percs, key = lambda x: x[1], reverse=True)[0][0]
        dominant_topics.append((i, dominant_topic))
        topic_percentages.append(topic_percs)
    return(dominant_topics, topic_percentages)

dominant_topics, topic_percentages = topics_per_document(model=lda_model, corpus=corpus, end=-1)

# Distribution of Dominant Topics in Each Document
df = pd.DataFrame(dominant_topics, columns=['Document_Id', 'Dominant_Topic'])
dominant_topic_in_each_doc = df.groupby('Dominant_Topic').size()
df_dominant_topic_in_each_doc = dominant_topic_in_each_doc.to_frame(name='count').reset_index()

# Total Topic Distribution by actual weight
topic_weightage_by_doc = pd.DataFrame([dict(t) for t in topic_percentages])
df_topic_weightage_by_doc = topic_weightage_by_doc.sum().to_frame(name='count').reset_index()

# Top 3 Keywords for each Topic
topic_top3words = [(i, topic) for i, topics in lda_model.show_topics(formatted=False)
                                 for j, (topic, wt) in enumerate(topics) if j < 3]

df_top3words_stacked = pd.DataFrame(topic_top3words, columns=['topic_id', 'words'])
df_top3words = df_top3words_stacked.groupby('topic_id').agg(', \n'.join)
df_top3words.reset_index(level=0,inplace=True)

ValueError: too many values to unpack (expected 3)

In [ ]:
import pandas as pd
print(pd.__version__)

In [22]:
%pip install pyLDAvis

Note: you may need to restart the kernel to use updated packages.


In [21]:
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

ModuleNotFoundError: No module named 'pyLDAvis.sklearn'

In [ ]:
pyLDAvis.save_html(vis, 'lda_visualization.html')


In [ ]:
# Ajouter une nouvelle colonne pour les étiquettes de sujets
tweets_df['Topic_Label'] = ""

# Mapper des noms de sujets aux identifiants de sujets
topic_labels_mapping = {
    0 : "Traitement et recherche sur le cancer",
    1 : "Combat contre le cancer et activisme",
    2 : "Santé et bien-être liés au cancer",
    3 : "Études cliniques et croissance tumorale",
    4 : "Effets des cannabinoïdes sur le cancer",
    5 : "Utilisation médicale du cannabis dans le traitement du cancer",
    6 : "Impact environnemental du traitement du cancer",
    7 : "Potentiel thérapeutique du cannabis dans la lutte contre le cancer"
}

# Assigner des étiquettes en fonction des sujets dominants
for index, row in df_dominant_topic.iterrows():
    tweet_index = row['Document_No']
    topic_id = row['Dominant_Topic']
    topic_label = topic_labels_mapping[topic_id]
    tweets_df.at[tweet_index, 'Topic_Label'] = topic_label


In [ ]:
# Spécifiez le chemin et le nom de fichier pour sauvegarder le DataFrame avec les étiquettes
output_file_path = "tweets_with_labels.csv"

# Enregistrez le DataFrame avec les étiquettes dans un fichier CSV
tweets_df.to_csv(output_file_path, sep=';', index=False)

# Confirmez que le fichier a été sauvegardé avec succès
print("Le DataFrame avec les étiquettes a été sauvegardé dans :", output_file_path)


In [ ]:
# Importez pandas
import pandas as pd

# Lisez le fichier CSV dans un DataFrame
t_df = pd.read_csv("tweets_with_labels.csv", sep = ";", skip_blank_lines=True)
# Affichez la forme du DataFrame
print(t_df.shape)

In [ ]:
null_values = t_df[t_df['Topic_Label'].isnull()]


In [ ]:
duplicates = t_df[t_df.duplicated(subset=['Topic_Label'], keep=False)]


In [ ]:
# Supprimer les lignes avec des valeurs nulles dans la colonne "Topic_Label"
#t_df = t_df.dropna(subset=['Topic_Label'])

# Supprimer les doublons dans la colonne "Topic_Label"
t_df = t_df.drop_duplicates(subset=['Fichier'], keep='first')


In [ ]:
print(t_df.shape)